<a href="https://colab.research.google.com/github/AshleyB1021/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [6]:
# Read the CSV file from AWS S3 bucket into a DataFrame
spark.sparkContext.addFile(url)
df = spark.read.csv("file://"+SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS Average_Price
    FROM home_sales
    WHERE Bedrooms = 4
    GROUP BY YEAR(Date)
""")

# Show the result
result.show()



+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
""")

# Show the result
result.show()




+---------+-------------+
|YearBuilt|Average_Price|
+---------+-------------+
|     2015|     288770.3|
|     2013|    295962.27|
|     2014|    290852.27|
|     2012|    293683.19|
|     2016|    290555.07|
|     2010|    292859.62|
|     2011|    291117.47|
|     2017|    292676.79|
+---------+-------------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
""")

# Show the result
result.show()



+---------+-------------+
|YearBuilt|Average_Price|
+---------+-------------+
|     2015|    297609.97|
|     2013|    303676.79|
|     2014|    298264.72|
|     2012|    307539.97|
|     2016|     293965.1|
|     2010|    285010.22|
|     2011|    276553.81|
|     2017|    280317.58|
+---------+-------------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")

# Show the result
result.show()



print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         31|    399856.95|
|         85|   1056336.74|
|         65|    736679.93|
|         53|     755214.8|
|         78|   1080649.37|
|         34|    401419.75|
|         81|   1053472.79|
|         28|    402124.62|
|         76|   1058802.78|
|         26|    401506.97|
|         27|    399537.66|
|         44|    400598.05|
|         12|    401501.32|
|         91|   1137372.73|
|         22|    402022.68|
|         93|   1026006.06|
|         47|     398447.5|
|          1|    401044.25|
|         52|    733780.26|
|         13|    398917.98|
+-----------+-------------+
only showing top 20 rows

--- 0.8932285308837891 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result_cached = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")

# Show the result
result_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         31|    399856.95|
|         85|   1056336.74|
|         65|    736679.93|
|         53|     755214.8|
|         78|   1080649.37|
|         34|    401419.75|
|         81|   1053472.79|
|         28|    402124.62|
|         76|   1058802.78|
|         26|    401506.97|
|         27|    399537.66|
|         44|    400598.05|
|         12|    401501.32|
|         91|   1137372.73|
|         22|    402022.68|
|         93|   1026006.06|
|         47|     398447.5|
|          1|    401044.25|
|         52|    733780.26|
|         13|    398917.98|
+-----------+-------------+
only showing top 20 rows

--- 2.547231912612915 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("path/to/partitioned_parquet_data")

In [18]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet("path/to/partitioned_parquet_data")

# Show the schema of the DataFrame
parquet_data.printSchema()

# Display the first few rows of the DataFrame
parquet_data.show()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|

In [19]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_sales")

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

result_parquet = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM parquet_sales
    WHERE price >= 350000
    GROUP BY view
""")

# Show the result
result_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         31|    399856.95|
|         85|   1056336.74|
|         65|    736679.93|
|         53|     755214.8|
|         78|   1080649.37|
|         34|    401419.75|
|         81|   1053472.79|
|         28|    402124.62|
|         76|   1058802.78|
|         26|    401506.97|
|         27|    399537.66|
|         44|    400598.05|
|         12|    401501.32|
|         91|   1137372.73|
|         22|    402022.68|
|         93|   1026006.06|
|         47|     398447.5|
|          1|    401044.25|
|         52|    733780.26|
|         13|    398917.98|
+-----------+-------------+
only showing top 20 rows

--- 0.9804155826568604 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [22]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("The home_sales temporary table is still cached.")
else:
    print("The home_sales temporary table is no longer cached.")


The home_sales temporary table is no longer cached.
